In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean

In [3]:
 # Start a Spark session
spark = SparkSession.builder \
    .appName("Crime Data Analysis") \
    .getOrCreate()

24/04/21 23:40:08 WARN Utils: Your hostname, Shwetas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.66.64.165 instead (on interface en0)
24/04/21 23:40:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 23:40:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/21 23:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/21 23:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# Load datasets
crimeData = spark.read.csv('crimeData.csv', header=True, inferSchema=True)
US_Social = spark.read.csv('US_unemploymentRate_PovertyRate.csv', header=True, inferSchema=True)

#crimeData = spark.read.csv('gs://bigdata-liya/crimeData.csv', header=True, inferSchema=True)
#US_Social = spark.read.csv('gs://bigdata-liya/US_unemploymentRate_PovertyRate.csv', header=True, inferSchema=True)





In [5]:
crimeData.show()

+----+-----------+--------------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+
|Year|agency_code|        County|population|violent_crimes|homicides|rapes|assaults|robberies|months_reported|crimes_percapita|homicides_percapita|rapes_percapita|assaults_percapita|robberies_percapita|
+----+-----------+--------------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+
|1975|    NM00101|    NEW MEXICO|    286238|          2383|       30|  181|    1353|      819|             12|          832.52|              10.48|          63.23|            472.68|             286.13|
|1975|    TX22001|         TEXAS|    112478|           278|        5|   28|     132|      113|             12|          247.16|               4.45|          24.89|            117.36|      

In [6]:
# Show basic stats
crimeData.describe().show()

# Filter data from 1990 to 2015
crime_data_filtered = crimeData.filter(crimeData["Year"] >= 1990)

# Inner join data
combined_data = crime_data_filtered.join(US_Social, ["Year", "County"], "inner")

24/04/21 23:40:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------+---------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+
|summary|              Year|agency_code|   County|        population|    violent_crimes|         homicides|             rapes|         assaults|         robberies|   months_reported| crimes_percapita|homicides_percapita|   rapes_percapita|assaults_percapita|robberies_percapita|
+-------+------------------+-----------+---------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+
|  count|              2829|       2788|     2829|              2760|              2794|              2795|              2754|             2753|              2754|

In [7]:
crime_data_filtered.show()

+----+-----------+--------------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+
|Year|agency_code|        County|population|violent_crimes|homicides|rapes|assaults|robberies|months_reported|crimes_percapita|homicides_percapita|rapes_percapita|assaults_percapita|robberies_percapita|
+----+-----------+--------------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+
|1990|    NM00101|    NEW MEXICO|    384736|          5121|       34|  222|    3835|     1030|             12|         1331.04|               8.84|           57.7|            996.79|             267.72|
|1990|    TX22001|         TEXAS|    261721|          1876|        8|  139|    1143|      586|             12|          716.79|               3.06|          53.11|            436.72|      

In [8]:
combined_data.show()

+----+----------+-----------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+-----------------+------------+
|Year|    County|agency_code|population|violent_crimes|homicides|rapes|assaults|robberies|months_reported|crimes_percapita|homicides_percapita|rapes_percapita|assaults_percapita|robberies_percapita|unemployment rate|Poverty Rate|
+----+----------+-----------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+-----------------+------------+
|1990|     TEXAS|    TX22001|    261721|          1876|        8|  139|    1143|      586|             12|          716.79|               3.06|          53.11|            436.72|              223.9|      6.383333333|        15.9|
|1990|     TEXAS|    TX22701|    465622|          3326|       46|  280|    1539|

In [9]:
# Calculating mean for each column and replacing missing values
means = combined_data.select([mean(c).alias(c) for c in combined_data.columns if combined_data.select(c).dtypes[0][1] != 'string'])
combined_data_filled = combined_data.na.fill(means.first().asDict())

In [10]:
combined_data_filled.show()

+----+----------+-----------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+-----------------+------------+
|Year|    County|agency_code|population|violent_crimes|homicides|rapes|assaults|robberies|months_reported|crimes_percapita|homicides_percapita|rapes_percapita|assaults_percapita|robberies_percapita|unemployment rate|Poverty Rate|
+----+----------+-----------+----------+--------------+---------+-----+--------+---------+---------------+----------------+-------------------+---------------+------------------+-------------------+-----------------+------------+
|1990|     TEXAS|    TX22001|    261721|          1876|        8|  139|    1143|      586|             12|          716.79|               3.06|          53.11|            436.72|              223.9|      6.383333333|        15.9|
|1990|     TEXAS|    TX22701|    465622|          3326|       46|  280|    1539|

In [11]:
# Assemble vectors (features)
assembler = VectorAssembler(inputCols=["homicides_percapita", "rapes_percapita", "assaults_percapita", "robberies_percapita", "unemployment rate", "Poverty Rate"], outputCol="features")
output = assembler.transform(combined_data_filled)
final_data = output.select("features", "crimes_percapita")

In [12]:
final_data.show()

+--------------------+----------------+
|            features|crimes_percapita|
+--------------------+----------------+
|[3.06,53.11,436.7...|          716.79|
|[9.88,60.13,330.5...|          714.31|
|[44.39,133.48,121...|         2438.23|
|[6.6,49.68,667.52...|          991.77|
|[3.43,13.33,49.64...|          128.62|
|[29.04,96.51,997....|         1748.36|
|[34.83,81.87,479....|          1388.3|
|[24.68,78.81,236....|         1000.16|
|[15.63,104.42,390...|          847.21|
|[22.22,45.94,238....|          612.22|
|[10.27,93.17,881....|          1507.0|
|[4.07,37.91,79.63...|          229.48|
|[77.77,49.93,1116...|         2458.23|
|[9.73,62.1,454.91...|          788.61|
|[10.3,58.04,228.7...|          624.11|
|[48.62,117.47,130...|         2568.26|
|[9.31,50.35,725.3...|         1066.99|
|[2.5,12.5,44.62,5...|          119.23|
|[42.65,96.68,1061...|         1949.82|
|[36.5,72.82,657.1...|         1599.93|
+--------------------+----------------+
only showing top 20 rows



In [13]:
# Standardization of features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(final_data)
final_data = scalerModel.transform(final_data)

# Split data
train_data, test_data = final_data.randomSplit([0.8, 0.2])

In [14]:
train_data.show()

+--------------------+----------------+--------------------+
|            features|crimes_percapita|      scaledFeatures|
+--------------------+----------------+--------------------+
|[0.21,3.21,1.61,1...|           16.49|[0.01559192386298...|
|[0.21,3.29,21.16,...|           61.02|[0.01559192386298...|
|[0.21,3.38,4.43,1...|           26.26|[0.01559192386298...|
|[0.62,5.65,24.15,...|            66.6|[0.04603329902404...|
|[0.68,31.49,85.81...|          219.09|[0.05048813441347...|
|[0.77,27.58,338.6...|          440.75|[0.05717038749760...|
|[0.83,4.26,13.51,...|            49.9|[0.06162522288703...|
|[0.91,14.43,36.31...|           87.95|[0.06756500340626...|
|[1.04,9.48,36.49,...|            87.0|[0.07721714675001...|
|[1.1,15.76,37.76,...|            92.1|[0.08167198213944...|
|[1.27,11.8,35.13,...|           88.41|[0.09429401574281...|
|[1.32,9.93,27.16,...|           71.45|[0.09800637856733...|
|[1.43,5.51,28.39,...|           78.01|[0.10617357678127...|
|[1.47,25.89,276.4...|  

In [31]:
# Linear Regression Model
lr = LinearRegression(featuresCol='scaledFeatures', labelCol='crimes_percapita')
lr_model = lr.fit(train_data)
train_results = lr_model.evaluate(train_data)

24/04/21 23:51:47 WARN Instrumentation: [063dd1ca] regParam is zero, which might cause numerical instability and overfitting.


In [16]:
# Print model metrics
print("Coefficients:", lr_model.coefficients)
print("Intercept:", lr_model.intercept)
print("RMSE:", train_results.rootMeanSquaredError)
print("R2:", train_results.r2)

Coefficients: [13.469636627624679,23.38179086038672,270.0061725583708,244.50913993249,-3.6956508284724155e-05,-0.0004633384685930735]
Intercept: 0.0021569268979068495
RMSE: 0.0063774663193577556
R2: 0.9999999998442911


In [17]:
# Define the model metrics
linReg_model = {
    "Coefficients": lr_model.coefficients,
    "Intercept": lr_model.intercept,
    "RMSE": train_results.rootMeanSquaredError,
    "R2": train_results.r2
}

# Print the model metrics dictionary
for key, value in linReg_model.items():
    print(f"{key}: {value}")


Coefficients: [13.468946072649999,23.38144704332784,270.00556639117775,244.51077450568474,-0.0006322602546153323,0.0003406095294259408]
Intercept: 0.001168113557108646
RMSE: 0.006215581503068614
R2: 0.9999999998480145


In [18]:
import csv

# Define the file path
csv_file_path = "linReg_model.csv"

# Write the model metrics to a CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Metric", "Value"])
    for key, value in linReg_model.items():
        writer.writerow([key, value])

print("Model metrics have been saved to", csv_file_path)

Model metrics have been saved to linReg_model.csv


In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

In [21]:
# Calculate the 75th percentile for crimes_percapita
threshold = combined_data_filled.approxQuantile("crimes_percapita", [0.75], 0.05)[0]

# Convert crimes_percapita to a binary label
binary_crime = combined_data_filled.withColumn("label", (col("crimes_percapita") > threshold).cast("integer"))

# Define the features to be used in the model
feature_columns = ["homicides_percapita", "rapes_percapita", "assaults_percapita", "robberies_percapita", "unemployment rate", "Poverty Rate"]

# Assemble the features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
binary_crime = assembler.transform(binary_crime)


# Scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
binary_crime = scaler.fit(binary_crime).transform(binary_crime)


# Split the data into training and test sets
train_data, test_data = binary_crime.randomSplit([0.7, 0.3])

# Initialize and fit the Logistic Regression model
log_rig = LogisticRegression(featuresCol='scaledFeatures', labelCol='label')
log_reg_model = log_rig.fit(train_data)

predictions = log_reg_model.transform(test_data)

# Evaluate model performance using ROC-AUC
evaluator = BinaryClassificationEvaluator()
roc_auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
print("Coefficients:", lr_model.coefficients)
print("Intercept:", lr_model.intercept)
print("ROC-AUC:", roc_auc)

## ODDS RATIO

# import numpy as np
# from pyspark.sql.functions import udf
# from pyspark.sql.types import DoubleType

# # Assume log_rig_model is your trained LogisticRegressionModel
# coefficients = log_reg_model.coefficients

# # Convert Spark DenseVector to a Python list
# coeff_list = coefficients.toArray().tolist()

# # Calculate the odds ratios
# odds_ratios = [np.exp(c) for c in coeff_list]

# # Feature names (assuming they are set according to your earlier vector assembler)
# feature_columns = ["homicides_percapita", "rapes_percapita", "assaults_percapita", "robberies_percapita", "unemployment rate", "Poverty Rate"]

# # Combine feature names and their corresponding odds ratios into a DataFrame
# odds_ratio_df = spark.createDataFrame(zip(feature_columns, odds_ratios), ["Feature", "Odds_Ratio"])

# Show the odds ratios for each feature
odds_ratio_df.show()




Coefficients: [13.468946072649999,23.38144704332784,270.00556639117775,244.51077450568474,-0.0006322602546153323,0.0003406095294259408]
Intercept: 0.001168113557108646
ROC-AUC: 1.0


NameError: name 'odds_ratio_df' is not defined

In [ ]:
import numpy as np
import pandas as pd

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType
coefficients = log_reg_model.coefficients
odds_ratios = [float(np.exp(c)) for c in coefficients.toArray()]
features_odds = list(zip(feature_columns, odds_ratios, coefficients.toArray()))
# Define schema
schema = StructType([
    StructField("Feature", StringType(), True),
    StructField("Odds_Ratio", DoubleType(), True),
    StructField("Coefficient", DoubleType(), True)
])

# Creating Spark DataFrame for output using schema
#results_spark_df = spark.createDataFrame(pd.DataFrame(features_odds, columns=["Feature", "Odds_Ratio", "Coefficient"]), schema=schema)


In [28]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Assemble vectors (features)
assembler = VectorAssembler(
    inputCols=["homicides_percapita", "rapes_percapita", "assaults_percapita", "robberies_percapita", "unemployment rate", "Poverty Rate"],
    outputCol="features"
)
final_data = assembler.transform(combined_data_filled)

# Standardization of features (optional but recommended for linear models, less critical for trees)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
final_data = scaler.fit(final_data).transform(final_data)

# Split data into training and testing sets
train_data, test_data = final_data.randomSplit([0.7, 0.3])


# Random Forest Model
rf = RandomForestRegressor(featuresCol='scaledFeatures', labelCol='crimes_percapita', numTrees=10)
rf_model = rf.fit(train_data)

# Predictions
predictions = rf_model.transform(test_data)
predictions.select("prediction", "crimes_percapita").show(5)


# Evaluate model
evaluator = RegressionEvaluator(labelCol="crimes_percapita", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data:", rmse)

evaluator.setMetricName("r2")
r2 = evaluator.evaluate(predictions)
print("R-squared on test data:", r2)


import pandas as pd


+------------------+----------------+
|        prediction|crimes_percapita|
+------------------+----------------+
| 646.1528408214882|          716.79|
|  635.029724773156|          714.31|
|2366.0393205128207|         2438.23|
| 2433.291166666667|         2458.23|
|1442.5388556443554|          1507.0|
+------------------+----------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 79.74568193154786
R-squared on test data: 0.9769840990621038


In [24]:

# Feature Importances
feature_importances = rf_model.featureImportances.toArray()
importance_df = pd.DataFrame(list(zip(assembler.getInputCols(), feature_importances)),
                             columns=["Feature", "Importance"])
importance_df 

,Feature,Importance
0,homicides_percapita,0.201543
1,rapes_percapita,0.070522
2,assaults_percapita,0.307424
3,robberies_percapita,0.345620
4,unemployment rate,0.033584
5,Poverty Rate,0.041307


In [25]:
import pandas as pd

# Define the model metrics for linear regression
linReg_model = {
    "Coefficients": lr_model.coefficients,
    "Intercept": lr_model.intercept,
    "RMSE": train_results.rootMeanSquaredError,
    "R2": train_results.r2
}

# Define the feature importances for random forest
feature_importances = rf_model.featureImportances.toArray()
importance_df = pd.DataFrame(list(zip(assembler.getInputCols(), feature_importances)),
                             columns=["Feature", "Importance"])

# Combine the model metrics and feature importances into a single DataFrame
combined_df = pd.DataFrame([linReg_model])
combined_df["homicides_percapita"] = importance_df.loc[0, "Importance"]
combined_df["assaults_percapita"] = importance_df.loc[2, "Importance"]
combined_df["robberies_percapita"] = importance_df.loc[3, "Importance"]
combined_df["unemployment_rate"] = importance_df.loc[4, "Importance"]
combined_df["poverty_rate"] = importance_df.loc[5, "Importance"]

# Reorder the columns
combined_df = combined_df[["R2", "RMSE", "homicides_percapita", "assaults_percapita", 
                           "robberies_percapita", "unemployment_rate", "poverty_rate"]]

# Write the combined DataFrame to a CSV file
combined_df.to_csv("model_results.csv", index=False)

print("Model results have been saved to model_results.csv")


Model results have been saved to model_results.csv


In [26]:
combined_df

,R2,RMSE,homicides_percapita,assaults_percapita,robberies_percapita,unemployment_rate,poverty_rate
0,1.0,0.006216,0.201543,0.307424,0.34562,0.033584,0.041307


In [27]:
linReg_model

{'Coefficients': DenseVector([13.4689, 23.3814, 270.0056, 244.5108, -0.0006, 0.0003]),
 'Intercept': 0.001168113557108646,
 'RMSE': 0.006215581503068614,
 'R2': 0.9999999998480145}

In [29]:
import pandas as pd

# Define the evaluation metrics and feature importances
evaluation_metrics = {
    "R2": r2,
    "RMSE": rmse
}

# Combine the evaluation metrics and feature importances into a single dictionary
combined_metrics = {**evaluation_metrics, **dict(importance_df.values)}

# Convert the combined metrics dictionary to a DataFrame
combined_metrics_df = pd.DataFrame.from_dict(combined_metrics, orient='index', columns=['Value'])

# Print the DataFrame
print(combined_metrics_df)


                         Value
R2                    0.976984
RMSE                 79.745682
homicides_percapita   0.201543
rapes_percapita       0.070522
assaults_percapita    0.307424
robberies_percapita   0.345620
unemployment rate     0.033584
Poverty Rate          0.041307


In [32]:
# Linear Regression Model
lr = LinearRegression(featuresCol='scaledFeatures', labelCol='crimes_percapita')
lr_model = lr.fit(train_data)

# Get the coefficient names from the input columns of the VectorAssembler
input_columns = assembler.getInputCols()

# Create a dictionary to store the model metrics including coefficient names
linReg_model = {
    "Intercept": lr_model.intercept
}

# Extract and store the coefficient values with their corresponding names
for idx, coef_value in enumerate(lr_model.coefficients):
    coefficient_name = input_columns[idx]
    linReg_model[coefficient_name] = coef_value

# Evaluate the model
train_results = lr_model.evaluate(train_data)

# Add RMSE and R2 to the model metrics dictionary
linReg_model["RMSE"] = train_results.rootMeanSquaredError
linReg_model["R2"] = train_results.r2

# Print the model metrics dictionary
for key, value in linReg_model.items():
    print(f"{key}: {value}")


24/04/21 23:52:44 WARN Instrumentation: [76f2517a] regParam is zero, which might cause numerical instability and overfitting.


Intercept: -0.000780754733371334
homicides_percapita: 13.470226947361162
rapes_percapita: 23.381964714757423
assaults_percapita: 270.00597477948577
robberies_percapita: 244.50886078307946
unemployment rate: 0.00011909312568863835
Poverty Rate: 5.517128334099256e-05
RMSE: 0.00632889243599336
R2: 0.999999999844241


In [33]:
import pandas as pd

# Define the evaluation metrics and feature importances for random forest
rf_metrics = {
    "R2": r2,
    "RMSE": rmse
}

# Combine the evaluation metrics and feature importances into a single dictionary for random forest
rf_combined_metrics = {**rf_metrics, **dict(importance_df.values)}

# Convert the combined metrics dictionary to a DataFrame for random forest
rf_combined_metrics_df = pd.DataFrame.from_dict(rf_combined_metrics, orient='index', columns=['Random_Forest'])

# Linear Regression Model
lr = LinearRegression(featuresCol='scaledFeatures', labelCol='crimes_percapita')
lr_model = lr.fit(train_data)

# Get the coefficient names from the input columns of the VectorAssembler
input_columns = assembler.getInputCols()

# Create a dictionary to store the model metrics including coefficient names for linear regression
linReg_model = {
    "Intercept": lr_model.intercept
}

# Extract and store the coefficient values with their corresponding names for linear regression
for idx, coef_value in enumerate(lr_model.coefficients):
    coefficient_name = input_columns[idx]
    linReg_model[coefficient_name] = coef_value

# Evaluate the model for linear regression
train_results = lr_model.evaluate(train_data)

# Add RMSE and R2 to the model metrics dictionary for linear regression
linReg_model["RMSE"] = train_results.rootMeanSquaredError
linReg_model["R2"] = train_results.r2

# Convert the model metrics dictionary to a DataFrame for linear regression
linReg_model_df = pd.DataFrame.from_dict(linReg_model, orient='index', columns=['Linear_Regression'])

# Combine both model results into a single DataFrame
combined_results_df = pd.concat([rf_combined_metrics_df.T, linReg_model_df.T])

# Print the combined DataFrame
print(combined_results_df)


24/04/21 23:55:04 WARN Instrumentation: [4a6bd3ca] regParam is zero, which might cause numerical instability and overfitting.


                         R2       RMSE  homicides_percapita  rapes_percapita  \
Random_Forest      0.976984  79.745682             0.201543         0.070522   
Linear_Regression  1.000000   0.006329            13.470227        23.381965   

                   assaults_percapita  robberies_percapita  unemployment rate  \
Random_Forest                0.307424             0.345620           0.033584   
Linear_Regression          270.005975           244.508861           0.000119   

                   Poverty Rate  Intercept  
Random_Forest          0.041307        NaN  
Linear_Regression      0.000055  -0.000781  


In [34]:
# Write the combined results DataFrame to a CSV file
combined_results_df.to_csv('combined_results.csv')

print("Combined results have been saved to combined_results.csv")


Combined results have been saved to combined_results.csv


24/04/22 02:13:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2002779 ms exceeds timeout 120000 ms
24/04/22 02:13:23 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/22 02:13:23 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 